# Real experiment evaluation

In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import os

%matplotlib notebook
%reload_ext autoreload
%autoreload 2

np.set_printoptions(precision=2)

In [ ]:
#print(os.listdir('experiments/robot_test/'))
in_model =  ['circle2_double.csv', 
             'circle3_triple.csv', 
             'clover.csv',
             'eight2_double.csv', 
             'rounds.csv', 
             'straight1.csv', 
             'straight2.csv', 
             'straight3.csv', 
             'straight4.csv', 
             'straight5.csv', 
             'straight6.csv', 
             'triangle_double.csv']
     
out_of_model = ['pentagone_double.csv', 
                'walking.csv', 
                'stopping.csv'
                'walking_circle1.csv', 
                'walking_circle2.csv', 
                'walking_circle3.csv']


# TODO: currently we have analyzed only in_model. 
names = in_model

anchorsfile = 'experiments/anchors.csv'

tango_system_id = 7585
rtt_system_id = 7592

In [ ]:
from evaluate_dataset import read_anchors_df
anchors_df = read_anchors_df(anchorsfile)
anchors_df.head()

# Distance filtering

## Resampling

In [ ]:
from evaluate_dataset import resample, read_dataset

raise ValueError('Comment this out if you really want to regenerate results.')

for name in names:
    datafile = 'experiments/robot_test/' + name
    datafile_name = datafile.split('.')[0]
    resample_name = datafile_name + '_resampled.pkl'
    print('Creating new results.'.format(resample_name))

    data_df = read_dataset(datafile, anchors_df)
    tmax = data_df.timestamp.max()
    tmin = data_df.timestamp.min()
    resampled_df_rtt = resample(data_df, t_range=[tmin, tmax], t_delta=0.5, t_window=1.0, system_id='RTT') 
    resampled_df_tango = resample(data_df, t_range=[tmin, tmax], t_delta=0.5, t_window=0.1, system_id='Tango') 
    resampled_df = pd.concat((resampled_df_rtt, resampled_df_tango), ignore_index=True)
    resampled_df.sort_values(["timestamp", "anchor_name"], inplace=True)
    resampled_df.to_pickle(resample_name)
    print('Saved as', resample_name)

## Find calibration vs. trajectory times

In [ ]:
%matplotlib notebook

# hard-coded time indices of datasets where automatic detection did not work. 

# We can do below either with the raw dat or with resampled data. 
use_raw = False # set True if you want to use raw data. Probably better to use resampled 


simplified = dict(
    resampled = { # [start_idx, end_idx] for each trajectory bit.
    'experiments/robot_test/circle3_triple.csv':[[67,155],[160, 250],[299,386]],
    'experiments/robot_test/clover.csv':[[37, 382]], 
    'experiments/robot_test/eight2_double.csv': [[40, 412]]
    }, 
    raw = {
    'experiments/robot_test/circle3_triple.csv':[[67,155],[160, 250],[299,386]],
    'experiments/robot_test/clover.csv':[[37, 382]], 
    'experiments/robot_test/eight2_double.csv': [[40, 412]]
    }
)

def read_correct_dataset(datafile, use_raw=False):
    from evaluate_dataset import add_gt_resampled
    from evaluate_dataset import apply_distance_gt, apply_name
    
    if use_raw:
        from evaluate_dataset import add_gt_raw
        data_df = read_dataset(datafile, anchors_df)
        data_df = add_gt_raw(data_df, t_window=0.1)
        data_df.loc[:, 'distance_tango'] = data_df.apply(lambda row: apply_distance_gt(row, anchors_df), axis=1)
    else:
        datafile_root = datafile.split('.')[0]
        resample_name = datafile_root + '_resampled.pkl'
        print('reading', resample_name)
        data_df = pd.read_pickle(resample_name)
        data_df = add_gt_resampled(data_df, anchors_df)
        data_df.loc[:, "anchor_name"] = data_df.apply(lambda row: apply_name(row, anchors_df), axis=1)
    return data_df
    
for datafile in sorted(simplified.keys()):
    continue
    # below was used to find above hardcoded values. 
    data_df = read_correct_dataset(datafile, use_raw)
    
    tango_df = data_df.loc[data_df.system_id=="Tango"]
    tango_df.loc[:, "length"] = get_length(tango_df)
    
    fig = plt.figure()
    plt.plot(tango_df.index, tango_df.length)
    plt.title(datafile)
    plt.ylim([0, 0.1])

In [ ]:
raise ValueError('Comment this out if you really want to regenerate results.')

%matplotlib inline
import json

from evaluate_dataset import find_start_times, find_end_times
from evaluate_dataset import get_length, find_calibration_data

use_rase=False

for name in names:
    
    # calibration_data will store the start and end times of the calibration dataset.
    # for example, calibration_data[10] = [t1, t2] means that for the trajectory
    # starting at index 10, the times between t1 and t2 are valid calibration times. 
    calibration_data = {} 
    
    datafile = 'experiments/robot_test/' + name
    datafile_root = datafile.split('.')[0]
    data_df = read_correct_dataset(datafile, use_raw)

    tango_df = data_df.loc[data_df.system_id=="Tango"]
    tango_df.loc[:, "length"] = get_length(tango_df, plot=False)
    print(len(tango_df.length))
    
    plt.figure()
    plt.plot(tango_df.timestamp, tango_df.length)
    plt.title(datafile_root)
    
    
    simplified_picked = simplified['raw'] if use_raw else simplified['resampled']

    if datafile in simplified_picked.keys():
        print('using simplified calculation for', datafile_root)
        
        times = tango_df.timestamp.values
        
        tuples = simplified_picked[datafile] # [start_move, end_move]
        start_indices = sorted([t[0] for t in tuples])
        end_indices = sorted([t[1] for t in tuples])
        
        calibration_data = {
            start_indices[0]: [times[0], times[start_indices[0]]],
        } 
        for s, e in zip(start_indices[1:], end_indices[:-1]):
            calibration_data[s] = [times[e], times[s]]
    else:
        start_times, start_indices = find_start_times(tango_df, plot=False)
        end_times, end_indices = find_end_times(tango_df, plot=False)
        calibration_data = find_calibration_data(tango_df, start_indices=start_indices, start_times=start_times)
        
    for idx, [start_time, end_time] in calibration_data.items():
        plt.plot([start_time, start_time], [tango_df.length.min(), tango_df.length.max()], 
                 color='green')
        plt.plot([end_time, end_time], [tango_df.length.min(), tango_df.length.max()], 
                 color='orange')
        
    timesfile = datafile_root + '_times.json'
    with open(timesfile, 'w+') as file:
        valid_data = {str(k):val for k, val in calibration_data.items()}
        json.dump(valid_data, file)
    print('saved as ', timesfile)

## Calibrate distances

In [ ]:
raise ValueError('Comment this out if you really want to regenerate results.')

from evaluate_dataset import apply_calibrate

for name in names:
    datafile = 'experiments/robot_test/' + name
    datafile_root = datafile.split('.')[0]
    data_df = read_correct_dataset(datafile, use_raw)
   
    timesfile = datafile_root + '_times.json'
    with open(timesfile, 'r') as file:
        valid_data = json.load(file)
        print(valid_data)
        calibration_data = {int(k): val for k, val in valid_data.items()}
    
    ########### Calibrate using the available 
    print('calibrating....')
    rtt_df = data_df[data_df.system_id=='RTT']
    for j, start_index in enumerate(sorted(calibration_data.keys())):
        start_time, end_time = calibration_data[start_index]
        
        # calculate offsets
        calib_dict = dict()
        for i, (anchor_name, df) in enumerate(rtt_df.groupby('anchor_name')):
            df = df[(df.timestamp >= start_time) & (df.timestamp <= end_time)]
            errors = df.distance.values - df.distance_tango.values
            calib_dict[anchor_name] = dict(mean=np.mean(errors),  median=np.median(errors))

        # calibrate distances
        data_df.loc[:, 'distance_mean_{}'.format(j)] = data_df.apply(lambda row: apply_calibrate(row, calib_dict, 'mean'), axis=1)
        data_df.loc[:, 'distance_median_{}'.format(j)] = data_df.apply(lambda row: apply_calibrate(row, calib_dict, 'median'), axis=1)
        
    # calculate offsets
    calib_dict = {} 
    for i, (anchor_name, df) in enumerate(rtt_df.groupby('anchor_name')):
        errors = df.distance.values - df.distance_tango.values
        calib_dict[anchor_name] = dict(all_mean=np.mean(errors), all_median=np.median(errors))
        
    # calibrate distances
    data_df.loc[:, 'distance_mean_all'.format(j)] = data_df.apply(lambda row: apply_calibrate(row, calib_dict, 'all_mean'), axis=1)
    data_df.loc[:, 'distance_median_all'.format(j)] = data_df.apply(lambda row: apply_calibrate(row, calib_dict, 'all_median'), axis=1)
    
    calibrate_name = datafile_root + '_calibrated.pkl'
    data_df.to_pickle(calibrate_name)
    print('saved as', calibrate_name)

## Calibrate using static positions

In [ ]:
# TODO use calib1-calib5 datasets for calibration. Not sure if we want to do this, 
# but it could be a more realistic calibration variant compared to the others.

# Plot results

### Run below cell first. Then, either inspect individual plots, or jumping down to regenerating all plots. 

In [ ]:
%matplotlib inline

from plotting_tools import make_dirs_safe
from plotting_tools import savefig
from itertools import cycle

name = names[10]

out_dir = 'experiments/robot_test/plots/'
make_dirs_safe(out_dir)
print('will save under', out_dir)

def read_plot_df(name):
    datafile = 'experiments/robot_test/' + name
    datafile_name = datafile.split('.')[0]
    calibrate_name = datafile_name + '_calibrated.pkl'


    data_df = pd.read_pickle(calibrate_name)
    print('read', calibrate_name)

    id_vars = [c for c in data_df.columns if c[:8]!='distance']
    plot_df = data_df.melt(id_vars=id_vars, 
                           var_name="distance_type", 
                           value_name="distance")
    plot_df = plot_df[plot_df.system_id=='RTT']
    plot_df.sort_values(['timestamp', 'anchor_name'], inplace=True)
    plot_df.reset_index(inplace=True, drop=True)
    plot_df.loc[:, 'distance'] = plot_df.distance.astype(np.float32)
    plot_df.loc[:, 'timestamp'] = plot_df.timestamp.astype(np.float32)
    
    return data_df, plot_df

data_df, plot_df = read_plot_df(name)
plot_df.head()

In [ ]:
from plotting_tools import plot_cdf_raw

def plot_cdfs(plot_df, filename=''):
    colors = sns.color_palette('deep')

    fig, axarr = plt.subplots(2, 4)
    fig.set_size_inches(15, 10)
    axarr = axarr.reshape((-1, ))
    for i, (anchor_name, df) in enumerate(plot_df.sort_values("anchor_name").groupby("anchor_name")):

        axarr[i].set_title(anchor_name)
        df = df.sort_values("timestamp")
        gt_df = df[df.distance_type=="distance_tango"]

        color_cycle = cycle(colors)
        for distance_type in sorted(df.distance_type.unique()):
            if distance_type == "distance_tango" or distance_type == "distance_gt":
                continue
            meas_df = df[df.distance_type==distance_type]

            np.testing.assert_allclose(meas_df.timestamp.values, gt_df.timestamp.values)

            errors = np.abs(meas_df.distance.values - gt_df.distance.values)
            plot_cdf_raw(errors, ax=axarr[i], color=next(color_cycle), label=distance_type)
            axarr[i].set_ylabel('')
    axarr[i].legend(loc='lower left', bbox_to_anchor=[1.0, 0])
    axarr[0].set_ylabel('cdf [-]')
    axarr[4].set_ylabel('cdf [-]')
    for j in range(i+1, len(axarr)):
        axarr[j].axis('off')
    [axarr[i].set_xlabel('absolute distance error [m]') for i in range(4, len(axarr))]

    if filename != '':
        savefig(fig, filename)

filename = '{}{}_cdfs.png'.format(out_dir, name.split('.')[0])
print(filename)
plot_cdfs(plot_df)

In [ ]:
from plotting_tools import plot_cdf_raw
from itertools import cycle

def plot_times(plot_df, filename=''):
    colors = sns.color_palette('deep')
    
    fig, axarr = plt.subplots(2, 4)
    fig.set_size_inches(15, 10)
    axarr = axarr.reshape((-1, ))
    for i, (anchor_name, df) in enumerate(plot_df.sort_values("anchor_name").groupby("anchor_name")):
        axarr[i].set_title(anchor_name)
        df = df.sort_values("timestamp")

        color_cycle = cycle(colors)
        for distance_type in sorted(df.distance_type.unique()):
            meas_df = df[df.distance_type==distance_type]
            axarr[i].plot(meas_df.timestamp.values, meas_df.distance.values, color=next(color_cycle), label=distance_type)
    axarr[i].legend(loc='lower left', bbox_to_anchor=[1.0, 0])
    for j in range(i+1, len(axarr)):
        axarr[j].axis('off')

    if filename != '':
        savefig(fig, filename)

plot_times(plot_df)

In [ ]:
from itertools import cycle

def plot_rssis(plot_df, filename=''):
    colors = sns.color_palette('deep')

    fig, axarr = plt.subplots(2, 4, sharey=True, sharex=True)
    fig.set_size_inches(15, 10)
    axarr = axarr.reshape((-1, ))
    for i, (anchor_name, df) in enumerate(plot_df.sort_values("anchor_name").groupby("anchor_name")):

        axarr[i].set_title(anchor_name)
        df = df.sort_values("timestamp")
        gt_df = df[df.distance_type=="distance_tango"]

        color_cycle = cycle(colors)
        meas_df = df[df.distance_type=="distance_median_all"]

        assert np.allclose(meas_df.timestamp.values, gt_df.timestamp.values)

        errors = np.abs(meas_df.distance.values - gt_df.distance.values)
        rssis = meas_df.rssi.values

        axarr[i].scatter(rssis, errors, alpha=0.2)
    for j in range(i+1, len(axarr)):
        axarr[j].axis('off')


    if filename != '':
        savefig(fig, filename)
        
plot_rssis(plot_df)

In [ ]:
def plot_tango_components(data_df, filename=''):
    fig = plt.figure()
    data = data_df[data_df.system_id=="Tango"]
    sns.scatterplot(data=data, x='timestamp', y='px', linewidth=0.0, label='x')
    sns.scatterplot(data=data, x='timestamp', y='py', linewidth=0.0, label='y')
    sns.scatterplot(data=data, x='timestamp', y='pz', linewidth=0.0, label='z')
    plt.legend()
    
    if filename != '':
        savefig(fig, filename)
        
plot_tango_components(data_df)

In [ ]:
def plot_tango_2d(data_df, filename=''):
    tango_df = data_df.loc[data_df.system_id=="Tango"]

    fig = plt.figure()
    sns.scatterplot(data=tango_df, x='px', y='py', hue='timestamp', linewidth=0.0)
    sns.scatterplot(data=anchors_df, x='px', y='py', hue='anchor_name', linewidth=0.0, style='system_id', legend=False)
    plt.arrow(0, 0, 1, 0, color='red', head_width=0.2)
    plt.arrow(0, 0, 0, 1, color='green', head_width=0.2)
    delta = 0.1
    for i, a in anchors_df.iterrows():
        plt.annotate(a.anchor_name, (a.px+delta, a.py+delta))
    plt.legend()
    plt.axis('equal')
    if filename != '':
        savefig(fig, filename)
    
plot_tango_2d(data_df, '')

## (can directly jump here) Plot all and save

In [ ]:
for name in names:
    if name != 'rounds.csv':
        continue
    data_df, plot_df = read_plot_df(name)
    
    pure_name = name.split('.')[0]
    
    skeleton = out_dir + '{}_{}.png'
    plot_cdfs(plot_df, skeleton.format('cdf', pure_name))
    plot_times(plot_df, skeleton.format('time', pure_name))
    plot_tango_components(data_df, skeleton.format('tango_components', pure_name))
    plot_tango_2d(data_df, skeleton.format('tango_2d', pure_name))
    
    plot_cdfs(plot_df, skeleton.format(pure_name, 'cdf'))
    plot_times(plot_df, skeleton.format(pure_name, 'time'))
    plot_tango_components(data_df, skeleton.format(pure_name, 'tango_components'))
    plot_tango_2d(data_df, skeleton.format(pure_name, 'tango_2d'))

# (in beta-stage) Position prediction

In [ ]:
from trajectory_creator import get_trajectory

print(datafile, simplified)
if datafile in simplified.keys():
    print('using simplified.')
    tuples = simplified[datafile]
    start_indices = [t[0] for t in tuples]
    end_indices = [t[1] for t in tuples]
    print(start_indices, end_indices)
else:
    start_times, start_indices = find_start_times(tango_df, plot=False)
    end_times, end_indices = find_end_times(tango_df, plot=False)

# TODO read the correct trajectory
name = 'circle2_double.csv'
datafile = 'experiments/robot_test/'
plot_df, data_df = read_plot_df(name)
tango_df = data_df[data_df.system_id=='Tango']

print(name)
trajectory = get_trajectory(name)

df_predicted = pd.DataFrame(index=range(len(tango_df)), columns=["px", "py", "pz", "timestamp", "model_timestamp"])
df_predicted.loc[:, "timestamp"] = tango_df.timestamp
df_predicted.reset_index(inplace=True, drop=True)


tango_df.loc[:, "length"] = get_length(tango_df)

previous_end = 0
for i in range(len(start_indices)):
    i0 = start_indices[i]
    iN = end_indices[i]
    
    indices = np.arange(i0, iN)
    lengths = tango_df.length.values[indices]
    
    distances = np.cumsum(lengths)
    times, _, _  = trajectory.get_times_from_distances(arbitrary_distances=distances, plot=False)
    if len(times) < len(distances):
        print('tried to go until {} but ended at {}'.format(len(distances), len(times)))
        iN = i0 + len(times)
    
    basis = trajectory.get_basis(times=times)
    points = trajectory.get_sampling_points(basis=basis)
    
    plt.figure()
    trajectory.plot(basis=basis)
    plt.scatter(points[0, :], points[1, :])
    plt.axis('equal')

    # TODO rotate and scale to fit the Tango measurements.
    print(i0, iN)
    
    df_predicted.loc[i0:iN-1, ["px", "py"]] = points.T
    df_predicted.loc[i0:iN-1, "model_timestamp"] = times
    
    # fill also the stationary points:
    df_predicted.loc[previous_end:i0, ["px", "py"]] = points.T[0]
    df_predicted.loc[iN:-1, ["px", "py"]] = points.T[-1]
    previous_end = iN
df_predicted.loc[:, "pz"] = 0.0

In [ ]:
plt.figure()
sns.scatterplot(data=df_predicted, x='px', y='py', hue='timestamp', 
               linewidth=0.0)
df_predicted.head()

# Additional processing 

## This used to be useful. Could probably be deleted at some point.

In [ ]:
from evaluate_dataset import add_median_raw, add_gt_raw

print(datafile)
data_df = read_dataset(datafile)
data_df.head()

# Our way

#data_df = add_median_raw(data_df)
#t1=time.time()
#data_df = add_gt_raw(data_df, t_window=0.1)

# Alternative way. 

### IMPORTANT: this is not centered. Our own implementation add_median_raw is centered. 
### However ours is much much slower...

import datetime
data_df.sort_values("timestamp", inplace=True)
datetimes = [datetime.datetime.fromtimestamp(t/1000.0) for t in data_df.timestamp]
data_df.index = [pd.Timestamp(datetime) for datetime in datetimes]
for anchor_id in data_df.anchor_id.unique():
    anchor_df = data_df[data_df.anchor_id==anchor_id]
    rolling_data = anchor_df['distance'].rolling('1s', min_periods=1, center=False)
    data_df.loc[data_df.anchor_id==anchor_id, "distance_mean"] = rolling_data.mean()
    data_df.loc[data_df.anchor_id==anchor_id, "distance_median"] = rolling_data.median() 
data_df.index = range(len(data_df))

In [ ]:
fg = sns.FacetGrid(data_df, col='anchor_name', col_wrap=4, col_order=sorted(data_df.anchor_name.unique()))
fg.map(plt.plot, 'seconds', 'distance')
fg.map(plt.plot, 'seconds', 'distance_gt')

def plot_distances(data_df):
    import itertools
    colors = itertools.cycle(plt.get_cmap('tab10').colors)
    rtt_ids = anchors_df[anchors_df.system_id==rtt_system_id].anchor_id.unique()
    fig, axs = plt.subplots(1, len(rtt_ids), sharey=True)
    fig.set_size_inches(15, 5)
    for ax, anchor_id in zip(axs, rtt_ids):
        color = next(colors)
        data = data_df[data_df.anchor_id==anchor_id]
        anchor_name = anchors_df.loc[anchors_df.anchor_id==anchor_id, 'anchor_name']
        ax.plot(data.seconds, data.distance_gt, linestyle=':', label=anchor_name, color=color)
        ax.plot(data.seconds, data.distance, linestyle='-', color=color)
        ax.set_ylim(0, 15)
    #sns.scatterplot(data=data_df[data_df.system_id==rtt_system_id], x='seconds', y='distance_error',hue='anchor_id')
plot_distances(data_df)

In [ ]:
parameters = {
    'experiments/robot_test/static1.csv':  np.array([0, 0, 0.2]), # on robot, facing window
    'experiments/robot_test/static2.csv':  np.array([0, 0, 0.1]), # in wooden thing, facing window
    'experiments/robot_test/static3.csv':  np.array([0, 0, 0.1]), # in wooden thing, facing wall
    'experiments/robot_test/static4.csv':  np.array([0, 0, 1.1]), # in wooden thing on stand, facing wall
    'experiments/robot_test/static5.csv':  np.array([0, 0, 1.1]), # in wooden thing on stand, facing wall
    'experiments/robot_test/static6.csv':  np.array([0, 0, 1.1]), # in wooden thing on stand, facing wall
    'experiments/robot_test/static7.csv':  np.array([0, 0, 0.2]), # in wooden thing on stand, facing wall
    'experiments/robot_test/static8.csv':  np.array([0, 0, 0.2]), # in wooden thing on stand, facing wall
}

for i in range(1, 9):
    datafile = 'experiments/robot_test/static{}.csv'.format(i)
    position = parameters[datafile]

    data_df = read_dataset(datafile)
    plot_distances(data_df)
    name = datafile.split('/')[-1]
    plt.suptitle(name)

In [ ]:
import os
folder = 'experiments/robot_test/'
file_list = os.listdir(folder)
plt.figure()

end_points_lines_room = np.array([
    [5.046, 5.615, 0.0],
    [4.869, 4.231, 0.0],
    [5.870, 1.830, 0.0],
    [4.254, 1.567, 0.0],
    [2.924, 1.867, 0.0],
    [1.198, 1.416, 0.0]]).T
start_point_room = np.array([1.034, 5.410, 0.0]).reshape((3, 1))
end_points_lines = convert_room_to_robot(end_points_lines_room)
start_point = convert_room_to_robot(start_point_room)

for datafile in sorted(file_list):
    if datafile[-4:] != '.csv':
        continue
    if (datafile[:8] != 'straight') and (datafile[:5] != 'calib') or (datafile[:10] == 'calibratio'):
        continue
    data_df = pd.read_csv(folder + datafile)
    #data_df.loc[:, 'seconds'] = (data_df.timestamp.values - data_df.timestamp.min()) / 1000. # in seconds
    data = data_df[data_df.system_id==tango_system_id]
    sns.scatterplot(data=data, x='px', y='py', label=datafile, linewidth=0.0)

plt.scatter(*start_point[:2], color='black', s=10.0)
for end_point in end_points_lines.T:
    print(end_point)
    plt.scatter(*end_point[:2], color='black', s=5.0)
    plt.plot([start_point[0], end_point[0]], [start_point[1], end_point[1]], color='black')
    
plt.axis('equal')
plt.legend(loc='lower left', bbox_to_anchor=[0.8, 0])
#plt.legend()
#fill_ground_truth(data_df, current_gt=position)

In [ ]:
for datafile in sorted(file_list):
    if datafile[-4:] != '.csv':
        continue
    if datafile[:6] == 'static' or datafile[:5] == 'calib':
        continue
    else:
        data_df = pd.read_csv(folder + datafile)
        data = data_df[data_df.system_id==tango_system_id]
        plt.figure()
        sns.scatterplot(data=data, x='px', y='py', label=datafile, linewidth=0.0)

In [ ]:
# Visualization of how find_start_times etc. works.
from evaluate_dataset import find_start_times, find_end_times

test_index = range(100)
test_df = pd.DataFrame(index=test_index, columns=["length", "timestamp"])
start_index_test = [10, 50] 
duration = 20
lengths = np.zeros(100)
for s_idx in start_index_test:
    lengths[s_idx:s_idx+duration] = 1.0
test_df["length"] = lengths
test_df["timestamp"] = test_index
        
start_times, start_indices = find_start_times(test_df)
end_times, end_indices = find_end_times(test_df)
np.testing.assert_allclose(start_indices, start_index_test)
plt.figure()
plt.plot(test_df.timestamp, test_df.length)
for s in start_times:
    plt.plot([s, s], [test_df.length.min(), test_df.length.max()], label='t0', color='orange')
for s in end_times:
    plt.plot([s, s], [test_df.length.min(), test_df.length.max()], label='t0', color='black')